In [2]:
"""
Neural Operator Examples with PyTorch
======================================
Comprehensive examples showing different neural operator architectures
and training workflows using the neuraloperator library.

Installation:
pip install neuraloperator torch
"""

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

# ============================================================================
# Example 1: Basic Fourier Neural Operator (FNO)
# ============================================================================

def example_1_basic_fno():
    """
    Basic FNO for 2D problems like Darcy Flow or solving PDEs.
    FNO learns mappings between function spaces in the Fourier domain.
    """
    from neuralop.models import FNO
    
    # Create FNO model
    model = FNO(
        n_modes=(32, 32),        # Number of Fourier modes to use
        hidden_channels=64,       # Hidden layer dimension
        in_channels=2,            # Input function channels (e.g., coordinates)
        out_channels=1,           # Output function channels (e.g., solution)
        n_layers=4                # Number of Fourier layers
    )
    
    # Example input: batch of 2D functions
    batch_size = 16
    resolution = 64
    x = torch.randn(batch_size, 2, resolution, resolution)
    
    # Forward pass
    output = model(x)
    print(f"FNO Input shape: {x.shape}")
    print(f"FNO Output shape: {output.shape}")
    
    return model


# ============================================================================
# Example 2: Tensorized FNO (TFNO) - Memory Efficient
# ============================================================================

def example_2_tfno():
    """
    TFNO uses Tucker factorization for parameter efficiency.
    Reduces parameters by ~95% while maintaining accuracy.
    """
    from neuralop.models import TFNO
    
    # Create TFNO model with Tucker factorization
    model = TFNO(
        n_modes=(32, 32),
        hidden_channels=64,
        in_channels=2,
        out_channels=1,
        factorization='tucker',   # Tucker decomposition
        implementation='factorized',  # Efficient implementation
        rank=0.05                 # Use only 5% of parameters
    )
    
    # Example usage
    x = torch.randn(16, 2, 64, 64)
    output = model(x)
    
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    print(f"TFNO Total parameters: {total_params:,}")
    print(f"TFNO Output shape: {output.shape}")
    
    return model


# ============================================================================
# Example 3: Complete Training Loop with Darcy Flow Dataset
# ============================================================================

def example_3_training_loop():
    """
    Complete training example using the built-in Darcy Flow dataset.
    Shows data loading, model creation, and training.
    """
    from neuralop.models import FNO
    from neuralop.data.datasets import load_darcy_flow_small
    from neuralop.training import Trainer
    from neuralop.losses import LpLoss
    
    # Load Darcy Flow dataset
    train_loader, test_loaders, data_processor = load_darcy_flow_small(
        n_train=1000,
        batch_size=32,
        n_tests=[100],
        test_resolutions=[32],
        test_batch_sizes=[32],
    )
    
    # Create model
    model = FNO(
        n_modes=(16, 16),
        hidden_channels=32,
        in_channels=3,   # x, y coordinates + input field
        out_channels=1   # solution field
    )
    
    # Setup optimizer and scheduler
    optimizer = Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler = StepLR(optimizer, step_size=100, gamma=0.5)
    
    # Define losses
    train_loss = LpLoss(d=2, p=2)
    eval_losses = {'l2': LpLoss(d=2, p=2)}
    
    # Create trainer
    trainer = Trainer(
        model=model,
        n_epochs=20,
        data_processor=data_processor,
        device='cuda' if torch.cuda.is_available() else 'cpu',
        verbose=True
    )
    
    # Train the model
    trainer.train(
        train_loader=train_loader,
        test_loaders=test_loaders,
        optimizer=optimizer,
        scheduler=scheduler,
        regularizer=False,
        training_loss=train_loss,
        eval_losses=eval_losses
    )
    
    # Save model
    model.save_checkpoint(save_folder='./checkpoints/', save_name='darcy_fno')
    print("Model saved successfully!")
    
    return model, trainer


# ============================================================================
# Example 4: Custom Training Loop (More Control)
# ============================================================================

def example_4_custom_training():
    """
    Manual training loop for more control over the training process.
    """
    from neuralop.models import FNO
    from neuralop.losses import LpLoss
    
    # Create model
    model = FNO(n_modes=(16, 16), hidden_channels=32, in_channels=2, out_channels=1)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    # Loss and optimizer
    criterion = LpLoss(d=2, p=2)
    optimizer = Adam(model.parameters(), lr=1e-3)
    
    # Dummy training data
    n_epochs = 10
    for epoch in range(n_epochs):
        model.train()
        
        # Generate dummy batch
        x = torch.randn(32, 2, 64, 64).to(device)
        y = torch.randn(32, 1, 64, 64).to(device)
        
        # Forward pass
        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred, y)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        if (epoch + 1) % 2 == 0:
            print(f"Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}")
    
    return model


# ============================================================================
# Example 5: Geometry-Informed Neural Operator (GINO)
# ============================================================================

def example_5_gino():
    """
    GINO for irregular geometries and unstructured meshes.
    Combines FNO with graph neural networks.
    """
    from neuralop.models import GINO
    
    # Create GINO model
    model = GINO(
        in_channels=3,           # Input features
        out_channels=1,          # Output features
        gno_coord_dim=2,         # 2D coordinates
        projection_channels=64,   # Projection dimension
        gno_radius=0.1,          # Neighborhood radius for graph
        n_layers=4
    )
    
    # Example with irregular points
    batch_size = 8
    n_points = 1000
    
    # Points at irregular locations
    x = torch.randn(batch_size, n_points, 3)  # (batch, points, features)
    pos = torch.rand(batch_size, n_points, 2)  # (batch, points, 2D coords)
    
    output = model(x, pos)
    print(f"GINO Input shape: {x.shape}")
    print(f"GINO Output shape: {output.shape}")
    
    return model


# ============================================================================
# Example 6: 1D Time-Series with FNO
# ============================================================================

def example_6_fno_1d_timeseries():
    """
    Using FNO for 1D problems like Burgers equation or time-series.
    """
    from neuralop.models import FNO
    
    # 1D FNO
    model = FNO(
        n_modes=(64,),           # 1D Fourier modes
        hidden_channels=64,
        in_channels=1,           # Single input channel
        out_channels=1,          # Single output channel
        n_layers=4
    )
    
    # Example 1D signal
    batch_size = 16
    time_steps = 256
    x = torch.randn(batch_size, 1, time_steps)
    
    output = model(x)
    print(f"1D FNO Input shape: {x.shape}")
    print(f"1D FNO Output shape: {output.shape}")
    
    return model


# ============================================================================
# Example 7: 3D FNO for Volumetric Data
# ============================================================================

def example_7_fno_3d():
    """
    3D FNO for volumetric problems like 3D fluid dynamics.
    """
    from neuralop.models import FNO
    
    # 3D FNO
    model = FNO(
        n_modes=(16, 16, 16),    # 3D Fourier modes
        hidden_channels=32,
        in_channels=4,           # e.g., velocity (3) + pressure (1)
        out_channels=4,
        n_layers=4
    )
    
    # Example 3D volume
    batch_size = 4
    resolution = 32
    x = torch.randn(batch_size, 4, resolution, resolution, resolution)
    
    output = model(x)
    print(f"3D FNO Input shape: {x.shape}")
    print(f"3D FNO Output shape: {output.shape}")
    
    return model


# ============================================================================
# Example 8: Model Inference and Super-Resolution
# ============================================================================

def example_8_super_resolution():
    """
    Neural operators are resolution-invariant!
    Train at one resolution, test at another.
    """
    from neuralop.models import FNO
    
    # Create and train model at low resolution
    model = FNO(
        n_modes=(16, 16),
        hidden_channels=64,
        in_channels=2,
        out_channels=1
    )
    
    # Train on 32x32 data
    x_train = torch.randn(16, 2, 32, 32)
    output_train = model(x_train)
    print(f"Training resolution: {x_train.shape}")
    
    # Test on 128x128 data (4x higher resolution!)
    model.eval()
    with torch.no_grad():
        x_test = torch.randn(4, 2, 128, 128)
        output_test = model(x_test)
    
    print(f"Testing resolution: {x_test.shape}")
    print(f"Super-resolution output: {output_test.shape}")
    print("✓ Same model works on different resolutions!")
    
    return model


# ============================================================================
# Example 9: Loading Pretrained Models
# ============================================================================

def example_9_load_checkpoint():
    """
    Save and load model checkpoints.
    """
    from neuralop.models import FNO
    
    # Create and save model
    model = FNO(n_modes=(16, 16), hidden_channels=64, in_channels=2, out_channels=1)
    model.save_checkpoint(save_folder='./checkpoints/', save_name='my_fno_model')
    print("Model saved!")
    
    # Load model later
    loaded_model = FNO.from_checkpoint(
        save_folder='./checkpoints/',
        save_name='my_fno_model'
    )
    print("Model loaded!")
    
    return loaded_model


# ============================================================================
# Example 10: Physics-Informed Loss
# ============================================================================

def example_10_physics_informed():
    """
    Combine data-driven learning with physics-based constraints.
    """
    from neuralop.models import FNO
    from neuralop.losses import LpLoss
    
    model = FNO(n_modes=(16, 16), hidden_channels=64, in_channels=2, out_channels=1)
    data_loss = LpLoss(d=2, p=2)
    
    def physics_loss(pred, x):
        """
        Example: enforce smoothness or conservation laws
        """
        # Compute gradients using autograd
        pred.requires_grad_(True)
        grad = torch.autograd.grad(pred.sum(), x, create_graph=True)[0]
        
        # Example: penalize large gradients (smoothness)
        smooth_loss = torch.mean(grad ** 2)
        return smooth_loss
    
    # Training step
    x = torch.randn(16, 2, 64, 64, requires_grad=True)
    y = torch.randn(16, 1, 64, 64)
    
    pred = model(x)
    
    # Combined loss
    loss_data = data_loss(pred, y)
    loss_physics = physics_loss(pred, x)
    total_loss = loss_data + 0.1 * loss_physics
    
    print(f"Data loss: {loss_data.item():.4f}")
    print(f"Physics loss: {loss_physics.item():.4f}")
    print(f"Total loss: {total_loss.item():.4f}")
    
    return model


# ============================================================================
# Main execution
# ============================================================================

if __name__ == "__main__":
    print("=" * 70)
    print("Neural Operator Examples with PyTorch")
    print("=" * 70)
    
    print("\n[1] Basic FNO")
    print("-" * 70)
    example_1_basic_fno()
    
    print("\n[2] Tensorized FNO (TFNO)")
    print("-" * 70)
    example_2_tfno()
    
    print("\n[4] Custom Training Loop")
    print("-" * 70)
    example_4_custom_training()
    
    print("\n[6] 1D FNO for Time-Series")
    print("-" * 70)
    example_6_fno_1d_timeseries()
    
    print("\n[7] 3D FNO for Volumetric Data")
    print("-" * 70)
    example_7_fno_3d()
    
    print("\n[8] Super-Resolution (Resolution Invariance)")
    print("-" * 70)
    example_8_super_resolution()
    
    print("\n[10] Physics-Informed Loss")
    print("-" * 70)
    example_10_physics_informed()
    
    print("\n" + "=" * 70)
    print("Examples completed! Check individual functions for details.")
    print("=" * 70)

Neural Operator Examples with PyTorch

[1] Basic FNO
----------------------------------------------------------------------
FNO Input shape: torch.Size([16, 2, 64, 64])
FNO Output shape: torch.Size([16, 1, 64, 64])

[2] Tensorized FNO (TFNO)
----------------------------------------------------------------------
TFNO Total parameters: 500,833
TFNO Output shape: torch.Size([16, 1, 64, 64])

[4] Custom Training Loop
----------------------------------------------------------------------
Epoch [2/10], Loss: 32.0094
Epoch [4/10], Loss: 32.0041
Epoch [6/10], Loss: 32.0028
Epoch [8/10], Loss: 32.0006
Epoch [10/10], Loss: 32.0047

[6] 1D FNO for Time-Series
----------------------------------------------------------------------
1D FNO Input shape: torch.Size([16, 1, 256])
1D FNO Output shape: torch.Size([16, 1, 256])

[7] 3D FNO for Volumetric Data
----------------------------------------------------------------------
3D FNO Input shape: torch.Size([4, 4, 32, 32, 32])
3D FNO Output shape: torch.